
##Data Wrangling for the miRNA expression dataset:
The following lines of code:
1. Concatenate the individual datasets into a single unified DataFrame.
2. Remove miRNA features (rows or columns, depending on structure) that contain no expression values across samples.
3. Prepare the cleaned dataset for downstream analysis, such as differential expression or unsupervised clustering.

The goal is to ensure data quality and consistency before proceeding with any statistical or machine learning tasks.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
%matplotlib inline
import pandas as pd

The first step involves reading the individual data sets obtained in the previous step.


In [ ]:
# Reading stage I-II data frame
df_stages_I_II = pd.read_csv('df_stages_I_II.csv', index_col='File_ID')
df_stages_I_II.shape

(437, 20)

In [ ]:
# Reading stage III-IV data frame
df_stages_III_IV = pd.read_csv('df_stages_III_IV.csv', index_col='File_ID')
df_stages_III_IV.shape

(410, 20)

In [ ]:
# Now we join each data frame to obtain the one that will be wrangled
df_mir_expression = pd.concat([df_stages_I_II, df_stages_III_IV], axis=0)
df_mir_expression.shape

(847, 20)

In [ ]:
df_mir_expression.info()

<class 'pandas.core.frame.DataFrame'>
Index: 847 entries, 3cd62167-7962-44ea-8923-6e9c7fc97807.mirbase21.isoforms.quantification.txt to bc1a0208-4ce8-4ce6-bf37-ee83e5a202e9.mirbase21.isoforms.quantification.txt
Data columns (total 19 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   hsa-mir-29a      847 non-null    float64
 1   hsa-mir-125b-1   847 non-null    float64
 2   hsa-mir-125b-2   847 non-null    float64
 3   hsa-mir-145      847 non-null    float64
 4   hsa-mir-149      847 non-null    float64
 5   hsa-mir-607-5p   847 non-null    float64
 6   hsa-mir-1246     847 non-null    float64
 7   hsa-mir-4488     847 non-null    float64
 8   hsa-mir-6777-5p  847 non-null    float64
 9   hsa-mir-492      847 non-null    float64
 10  hsa-mir-200a     847 non-null    float64
 11  hsa-mir-338      847 non-null    float64
 12  hsa-mir-29c      847 non-null    float64
 13  hsa-mir-101      847 non-null    float64
 14  hsa-mir-148a   

Some miRNA will not be found in the transcriptome files. To know which miRNAs are missed, we will check the columns that show only 0s transforming them into strings and counting values. This way we can differentiate between miRNAs that don't express across some individuals and miRNA features that are not found in the files at all.

In [ ]:
# Select the miRNA columns which data type is float
columns = df_mir_expression.columns
float_columns = columns[:18]
# Pick and change to string data type each column
df_mir_expression[float_columns] = df_mir_expression[float_columns].astype('str')
df_mir_expression.head()

,hsa-mir-29a,hsa-mir-125b-1,hsa-mir-125b-2,hsa-mir-145,hsa-mir-149,hsa-mir-607-5p,hsa-mir-1246,hsa-mir-4488,hsa-mir-6777-5p,hsa-mir-492,hsa-mir-200a,hsa-mir-338,hsa-mir-29c,hsa-mir-101,hsa-mir-148a,hsa-mir-92a,hsa-mir-424,hsa-mir-210,Stages
File_ID,,,,,,,,,,,,,,,,,,,
3cd62167-7962-44ea-8923-6e9c7fc97807.mirbase21.isoforms.quantification.txt,6433.474746,96.445011,100.828873,2455.025573,3.704676,0.0,0.0,0.0,0.0,0.061745,1066.081631,147.754741,772.6714639999999,0.0,17596.706500999997,0.0,258.401,290.508173,Stage I-II
ef4cd175-6f73-4360-b2c6-71b424d64f53.mirbase21.isoforms.quantification.txt,6301.933109,79.897088,79.762807,1111.710899,5.908356,0.0,0.268562,0.0,0.0,0.0,2083.501186,975.415869,323.885335,0.0,120874.62469,0.0,522.889509,1422.839555,Stage I-II
98694eb1-1282-4426-8fb2-001ac8190323.mirbase21.isoforms.quantification.txt,5633.065554000001,87.07883100000001,91.506569,2537.58534,1.967884,0.0,0.491971,0.0,0.0,0.491971,3693.224731,139.719706,453.105099,0.0,62440.932759,0.0,175.141602,719.753268,Stage I-II
a6f1d4ee-b216-4b96-95a6-5705662254d7.mirbase21.isoforms.quantification.txt,17290.155720000002,151.468017,158.525667,3408.301859,54.56106,0.0,0.0,0.0,0.0,0.0,3575.242417,388.7136210000001,1074.934321,0.0,95548.36037,0.0,130.837964,69.490703,Stage I-II
e3f4c57a-45e8-4dd6-96b1-e12ba2bdb415.mirbase21.isoforms.quantification.txt,9060.427026,85.35494,81.798484,1135.93201,4.267747,0.0,0.0,0.0,0.0,0.0,2634.622522,522.087724,1532.832487,0.0,180015.705306,0.0,103.848509,396.189185,Stage I-II


In [ ]:
# Selecting miRNA features that showed 0s in the first 5 rows
mir_columns_0 = ['hsa-mir-607-5p', 'hsa-mir-4488', 'hsa-mir-6777-5p', 'hsa-mir-101', 'hsa-mir-92a']
# Creating a loop to check which feature has only 0s
for column in mir_columns_0:
 print(column)
 print(df_mir_expression[column].value_counts())
 print('')

hsa-mir-607-5p
hsa-mir-607-5p
0.0    847
Name: count, dtype: int64

hsa-mir-4488
hsa-mir-4488
0.0         802
0.417769      2
0.34794       1
0.121099      1
1.097843      1
0.28529       1
0.281826      1
0.757528      1
0.412994      1
0.169698      1
0.488347      1
0.219731      1
0.432478      1
0.175449      1
0.097405      1
0.285033      1
0.146162      1
1.240312      1
0.264744      1
0.566536      1
0.15282       1
0.283425      1
0.313375      1
0.660037      1
0.364201      1
0.219321      1
0.736909      1
0.16427       1
0.399078      1
0.234133      1
0.287578      1
1.178362      1
2.526548      1
0.297956      1
0.118052      1
0.736796      1
1.798319      1
0.199524      1
0.707442      1
0.316306      1
1.11507       1
0.150503      1
0.094435      1
0.250206      1
0.328186      1
Name: count, dtype: int64

hsa-mir-6777-5p
hsa-mir-6777-5p
0.0    847
Name: count, dtype: int64

hsa-mir-101
hsa-mir-101
0.0    847
Name: count, dtype: int64

hsa-mir-92a
hsa-mir-92a
0.0

In [ ]:
# Dropping the columns that are missed from the trancriptome files
mir_columns_to_drop = ['hsa-mir-607-5p', 'hsa-mir-6777-5p', 'hsa-mir-101', 'hsa-mir-92a']
df_mir_expression.drop(mir_columns_to_drop, axis=1, inplace=True)

In [ ]:
df_mir_expression.shape

(847, 15)

In [ ]:
# Returning to float data type the remaining columns
all_columns = df_mir_expression.columns
string_columns = all_columns[:14]
df_mir_expression[string_columns] = df_mir_expression[string_columns].astype('float')
df_mir_expression.info()

<class 'pandas.core.frame.DataFrame'>
Index: 847 entries, 3cd62167-7962-44ea-8923-6e9c7fc97807.mirbase21.isoforms.quantification.txt to bc1a0208-4ce8-4ce6-bf37-ee83e5a202e9.mirbase21.isoforms.quantification.txt
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   hsa-mir-29a     847 non-null    float64
 1   hsa-mir-125b-1  847 non-null    float64
 2   hsa-mir-125b-2  847 non-null    float64
 3   hsa-mir-145     847 non-null    float64
 4   hsa-mir-149     847 non-null    float64
 5   hsa-mir-1246    847 non-null    float64
 6   hsa-mir-4488    847 non-null    float64
 7   hsa-mir-492     847 non-null    float64
 8   hsa-mir-200a    847 non-null    float64
 9   hsa-mir-338     847 non-null    float64
 10  hsa-mir-29c     847 non-null    float64
 11  hsa-mir-148a    847 non-null    float64
 12  hsa-mir-424     847 non-null    float64
 13  hsa-mir-210     847 non-null    float64
 14  Stages          847 non-null   

The final step involves preparing the data for the subsequent steps rounding to 3 decimals each float value

In [ ]:
# Rounding all the miRNA expression columns to 3 decimals
round_columns = df_mir_expression.columns
round_columns = round_columns[:14]
for column in round_columns:
  df_mir_expression[column] = np.round(df_mir_expression[column], decimals=3)

In [ ]:
df_mir_expression.head()

,hsa-mir-29a,hsa-mir-125b-1,hsa-mir-125b-2,hsa-mir-145,hsa-mir-149,hsa-mir-1246,hsa-mir-4488,hsa-mir-492,hsa-mir-200a,hsa-mir-338,hsa-mir-29c,hsa-mir-148a,hsa-mir-424,hsa-mir-210,Stages
File_ID,,,,,,,,,,,,,,,
3cd62167-7962-44ea-8923-6e9c7fc97807.mirbase21.isoforms.quantification.txt,-0.288,-0.398,-0.394,-0.090,-0.230,-0.279,-0.168,-0.235,-0.761,-0.517,-0.009,-0.801,-0.087,-0.552,Stage I-II
ef4cd175-6f73-4360-b2c6-71b424d64f53.mirbase21.isoforms.quantification.txt,-0.317,-0.442,-0.447,-0.408,-0.220,-0.113,-0.168,-0.330,-0.360,0.985,-0.648,1.406,0.211,0.121,Stage I-II
98694eb1-1282-4426-8fb2-001ac8190323.mirbase21.isoforms.quantification.txt,-0.466,-0.423,-0.417,-0.070,-0.238,0.025,-0.168,0.431,0.274,-0.531,-0.464,0.157,-0.181,-0.297,Stage I-II
a6f1d4ee-b216-4b96-95a6-5705662254d7.mirbase21.isoforms.quantification.txt,2.127,-0.250,-0.248,0.136,-0.002,-0.279,-0.168,-0.330,0.228,-0.080,0.420,0.865,-0.231,-0.683,Stage I-II
e3f4c57a-45e8-4dd6-96b1-e12ba2bdb415.mirbase21.isoforms.quantification.txt,0.297,-0.427,-0.442,-0.402,-0.228,-0.279,-0.168,-0.330,-0.143,0.162,1.071,2.670,-0.261,-0.489,Stage I-II


In [ ]:
df_mir_expression.to_csv('df_mir_expression_wrangled.csv')
from google.colab import files
# Download the wrangled data frame
files.download('df_mir_expression_wrangled.csv')